# User Data Pre-Processing

- variable meanings are documented in shq_variables.html
- The following codes are used to remove invalid rows and clean up typos

In [ ]:
# import library and show all the rows when print out the whole dataframe
import pandas as pd
import seaborn as sns
pd.set_option('display.max_rows', None)

### Revert Json to CSV

In [ ]:
#df = pd.read_json(r'2019-12-16-latest-users.json',lines=True)
#df.to_csv(r'2019-12-16-latest-users.csv', index = None)

### Raw Data Info

In [ ]:
user_df = pd.read_csv('2019-12-16-latest-users.csv')
user_df.head()

<IPython.core.display.Javascript object>

,uuid,stored_at,activity_recent,activityrecent,age,education,gender,hand,home_environment,location,navigating_skills,sleep,travel_time
0,66857759-5d93-4635-914f-b643e34a7519,2018-04-11 13:37:03+00:00,NaN,NaN,16.0,high-school,m,right,suburbs,US,very-good,6.0,30-mins-to-1-hour
1,2d51b872-d192-4392-851e-e693d4576980,2016-06-22 07:43:52+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01d4e1af-2691-4312-b3f2-9b27c28e7578,2016-05-16 11:27:24+00:00,NaN,NaN,99.0,NaN,m,NaN,NaN,LB,NaN,NaN,NaN
3,30a73f52-9bfd-4310-b284-58a896428c0a,2017-08-30 22:47:44+00:00,NaN,NaN,68.0,college,f,right,city,CA,bad,8.0,less-30-mins
4,e1cecc23-b523-42f8-9b58-520ae053b48e,2016-07-31 23:36:48+00:00,NaN,NaN,18.0,high-school,m,left,mixed,SK,good,7.0,less-30-mins


In [ ]:
user_df.info()

### Drop useless columns

In [ ]:
# drop the variables useless to the demographical analysis
user_drop = user_df.copy() #store check points
user_drop = user_drop.drop(['stored_at','activity_recent','activityrecent'],axis=1)
user_drop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3936867 entries, 0 to 3936866
Data columns (total 10 columns):
 #   Column             Dtype  
---  ------             -----  
 0   uuid               object 
 1   age                float64
 2   education          object 
 3   gender             object 
 4   hand               object 
 5   home_environment   object 
 6   location           object 
 7   navigating_skills  object 
 8   sleep              float64
 9   travel_time        object 
dtypes: float64(2), object(8)
memory usage: 300.4+ MB


### Drop rows with NA

In [ ]:
user_drop = user_drop.dropna()

In [ ]:
user_drop.info()

In [ ]:
user_drop.describe()

- **As shown in the summary table above, there are some invalid age values and sleep times**
- **Given that the minimum age value can be input is 16, 16 is also a suspicious value for age**
- **people older than 70 but also into playing video game are not representative**
- **select age range 18 - 70**
- **daily sleep hour larger than 13 hours is suspicious**

In [ ]:
user_clean = user_drop.copy()

In [ ]:
user_clean = user_clean[user_clean['age'] <= 70]
user_clean = user_clean[user_clean['age'] >= 18 ]
user_clean = user_clean[user_clean['sleep'] < 13 ]

In [ ]:
sns.histplot(data=user_clean,x="age",binwidth = 5)

In [ ]:
user_clean.shape[0]

In [ ]:
user_clean["home_environment"].value_counts()/user_clean.shape[0]

In [ ]:
user_clean["gender"].value_counts()/user_clean.shape[0]

In [ ]:
user_clean["hand"].value_counts()/user_clean.shape[0]

In [ ]:
user_clean["education"].value_counts()/user_clean.shape[0]

university     0.368326
high-school    0.288130
college        0.246835
unspecified    0.050057
no-formal      0.046651
Name: education, dtype: float64

In [ ]:
user_clean["location"].value_counts()/user_clean.shape[0]

In [ ]:
user_clean["navigating_skills"].value_counts()/user_clean.shape[0]

In [ ]:
user_clean["travel_time"].value_counts()/user_clean.shape[0]

- **As shown above, there are some typos**

### Clean up typos

In [ ]:
user_clean['education'] = user_clean['education'].apply(lambda x: 'college' if x=='College' else x)
user_clean['gender'] = user_clean['gender'].apply(lambda x: 'm' if x=='M' else x)
user_clean['home_environment'] = user_clean['home_environment'].apply(lambda x: 'city' if x=='City' else x)
user_clean['travel_time'] = user_clean['travel_time'].apply(lambda x: 'less-30-mins' if x=='Less than 30 minutes' else x)

### Remove roles with useless levels

In [ ]:
# remove rows where gender == o
user_clean = user_clean[user_clean['gender'] != 'o']

# remove rows where education == unspecified
user_clean = user_clean[user_clean['education'] != 'unspecified']

In [ ]:
user_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 909456 entries, 3 to 3936863
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   uuid               909456 non-null  object 
 1   age                909456 non-null  float64
 2   education          909456 non-null  object 
 3   gender             909456 non-null  object 
 4   hand               909456 non-null  object 
 5   home_environment   909456 non-null  object 
 6   location           909456 non-null  object 
 7   navigating_skills  909456 non-null  object 
 8   sleep              909456 non-null  float64
 9   travel_time        909456 non-null  object 
dtypes: float64(2), object(8)
memory usage: 76.3+ MB


In [ ]:
user_clean["gender"].value_counts()/user_clean.shape[0]

m    0.52062
f    0.47938
Name: gender, dtype: float64

In [ ]:
user_clean["education"].value_counts()/user_clean.shape[0]

### Explore CSV File

In [ ]:
user_clean.to_csv("2019user_cleaned.csv",index=False)